In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

# Selenium WebDriver 설정 (Chrome 예시)
options = Options()
options.add_argument("--headless")  # 화면 표시 없이 실행 (필요시)
service = Service('chromedriver')   # chromedriver 경로
driver = webdriver.Chrome(service=service, options=options)

# 크롤링 대상 카테고리 목록 (카테고리 페이지 URL 또는 네비게이션용 이름)
categories = [
    {"name": "스킨/토너", "url": "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000100130001"},
    {"name": "에센스/세럼/앰플", "url": "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000100130002"},
    {"name": "크림", "url": "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000100130003"},
    {"name": "아이크림", "url": "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000100130004"}
]
# ※ URL은 예시입니다. 실제 dispCatNo는 사이트에서 확인해야 합니다.

# 결과 저장용 리스트 초기화
data = []  # 각 원소는 [브랜드명, 상품명, 리뷰내용]

for category in categories:
    # 1. 카테고리 페이지 접근
    driver.get(category["url"])
    time.sleep(2)  # 페이지 로드 대기 (필요에 따라 WebDriverWait으로 개선 가능)
    
    # 2. 상품 리스트 추출 (상품 링크 요소 찾기)
    product_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='goods/getGoodsDetail.do']")  # 상품 상세로 가는 링크들
    product_urls = [link.get_attribute('href') for link in product_links]
    
    # 각 상품 상세 페이지 순회
    for product_url in product_urls:
        driver.get(product_url)
        time.sleep(2)  # 상품 페이지 로드 대기
        
        # 상품의 브랜드명과 상품명 추출
        try:
            brand_name = driver.find_element(By.CSS_SELECTOR, "#moveBrandShop").text
        except:
            brand_name = ""
        try:
            product_name = driver.find_element(By.CSS_SELECTOR, "p.prd_name").text
        except:
            product_name = ""
        
        # 리뷰 탭 클릭 (리뷰 목록 로드)
        try:
            review_tab = driver.find_element(By.XPATH, "//a[contains(text(), '리뷰')]")
            review_tab.click()
        except:
            # 리뷰 탭이 없는 경우 (리뷰 없음 등) -> 다음 상품으로 넘어감
            continue
        time.sleep(2)  # 리뷰 탭 콘텐츠 로드 대기
        
        # 리뷰 목록 가져오기
        review_items = driver.find_elements(By.CSS_SELECTOR, "#gdasList > li")
        if not review_items:
            # 리뷰가 하나도 없는 상품 -> 스킵
            continue
        
        for item in review_items:
            # 각 리뷰 내용 추출
            try:
                review_text = item.find_element(By.CSS_SELECTOR, "div.review_cont > p").text
            except:
                # 리뷰 텍스트가 특정 태그 안에 있을 경우 없으면 다음으로
                try:
                    review_text = item.find_element(By.CSS_SELECTOR, "div.review_cont").text
                except:
                    review_text = ""
            if review_text:
                # 브랜드명, 상품명, 리뷰내용을 하나의 행으로 기록
                data.append([brand_name, product_name, review_text])
    # (선택) 카테고리별 진행 상황 출력
    print(f"카테고리 '{category['name']}' 처리 완료 - 수집 리뷰 수: {len(data)}")

# 3. 수집 결과를 데이터프레임으로 변환 후 CSV/XLSX 저장
df = pd.DataFrame(data, columns=["브랜드명", "상품명", "리뷰내용"])
df.to_csv("oliveyoung_reviews.csv", index=False, encoding="utf-8-sig")
df.to_excel("oliveyoung_reviews.xlsx", index=False)
print("크롤링 완료 – CSV와 XLSX 파일로 저장되었습니다.")


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
